<a href="https://colab.research.google.com/github/faizan-codes/fine-tuned-models/blob/main/Mistral_7B_LoRA_Fine_tuning_on_Alpaca_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mistral-7B LoRA Fine-tuning on Alpaca Dataset - Complete Implementation
# This notebook provides a complete setup for fine-tuning Mistral-7B using LoRA on 1% of the Alpaca dataset


In [1]:
# Step 1: Install Required Libraries
print("Step 1: Installing required libraries...")
!pip install -qU transformers
!pip install -qU peft
!pip install -qU datasets
!pip install -qU accelerate
!pip install -qU bitsandbytes
!pip install -qU trl
!pip install -qU wandb
!pip install -qU torch torchvision torchaudio

Step 1: Installing required libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
 

In [2]:
# Step 2: Import Libraries
print("Step 2: Importing libraries...")
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from datasets import load_dataset
import numpy as np
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

Step 2: Importing libraries...


In [3]:
# Step 3: Check GPU and Setup
print("Step 3: Checking GPU availability...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


Step 3: Checking GPU availability...
Using device: cuda
GPU: Tesla T4
Memory: 14.7 GB


In [4]:
# Step 4: Configuration
print("Step 4: Setting up configuration...")
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
DATASET_NAME = "tatsu-lab/alpaca"
OUTPUT_DIR = "./mistral-7b-alpaca-lora"
DATASET_SIZE = 0.01  # 1% of the dataset

# LoRA Configuration
lora_config = LoraConfig(
    r=16,                    # Rank - lower means fewer parameters
    lora_alpha=32,          # LoRA scaling parameter
    target_modules=[        # Which modules to apply LoRA to
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head"
    ],
    lora_dropout=0.1,       # Dropout for LoRA layers
    bias="none",            # No bias terms
    task_type=TaskType.CAUSAL_LM,
)

# Quantization Configuration for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

Step 4: Setting up configuration...


In [5]:
# Step 5: Load and Prepare Dataset
print("Step 5: Loading and preparing dataset...")
dataset = load_dataset(DATASET_NAME)
print(f"Original dataset size: {len(dataset['train'])}")

# Take 1% of the dataset
train_size = int(len(dataset['train']) * DATASET_SIZE)
train_dataset = dataset['train'].select(range(train_size))
print(f"Using {train_size} samples for training")

Step 5: Loading and preparing dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Original dataset size: 52002
Using 520 samples for training


In [6]:
# Step 6: Format Dataset for Instruction Following
print("Step 6: Formatting dataset...")
def format_instruction(example):
    """Format the Alpaca dataset for instruction following"""
    instruction = example['instruction']
    input_text = example['input']
    output = example['output']

    if input_text:
        prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    else:
        prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n{output}"

    return {"text": prompt}

# Apply formatting
formatted_dataset = train_dataset.map(format_instruction, remove_columns=train_dataset.column_names)
print("Dataset formatted successfully!")

Step 6: Formatting dataset...


Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Dataset formatted successfully!


In [7]:
# Step 7: Load Tokenizer
print("Step 7: Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenizer.padding_side = "right"  # Fix padding side

Step 7: Loading tokenizer...


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
# Step 8: Load Model with Quantization
print("Step 8: Loading model with quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

Step 8: Loading model with quantization...


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
# Step 9: Apply LoRA
print("Step 9: Applying LoRA configuration...")
model = get_peft_model(model, lora_config)

# Print trainable parameters
trainable_params = 0
all_param = 0
for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()

print(f"Trainable params: {trainable_params:,} || All params: {all_param:,} || Trainable%: {100 * trainable_params / all_param:.4f}")


Step 9: Applying LoRA configuration...
Trainable params: 42,520,576 || All params: 3,794,591,744 || Trainable%: 1.1206


In [10]:
# Step 10: Training Arguments
print("Step 10: Setting up training arguments...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    save_strategy="steps",
    save_total_limit=3,
    push_to_hub=False,
    disable_tqdm=False,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    label_names=["labels"]
)

Step 10: Setting up training arguments...


In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=training_args,
    # packing=False,
)


Adding EOS to train dataset:   0%|          | 0/520 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/520 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/520 [00:00<?, ? examples/s]

In [12]:
# Step 12: Start Training
print("Step 12: Starting training...")
print("This may take some time depending on your GPU...")
trainer.train()

Step 12: Starting training...
This may take some time depending on your GPU...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
25,1.650800
50,1.092200
75,0.896700


TrainOutput(global_step=99, training_loss=1.120135374743529, metrics={'train_runtime': 1405.0937, 'train_samples_per_second': 1.11, 'train_steps_per_second': 0.07, 'total_flos': 8200978576834560.0, 'train_loss': 1.120135374743529})

In [13]:
# Step 13: Save the Fine-tuned Model
print("Step 13: Saving the fine-tuned model...")
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"Model saved to {OUTPUT_DIR}")

Step 13: Saving the fine-tuned model...
Model saved to ./mistral-7b-alpaca-lora


In [14]:
# Step 14: Test the Fine-tuned Model
print("Step 14: Testing the fine-tuned model...")

# Load the fine-tuned model for inference
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# Load the LoRA adapter
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)

def generate_response(instruction, input_text="", max_length=256):
    """Generate response using the fine-tuned model"""
    if input_text:
        prompt = f"Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n"
    else:
        prompt = f"Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            top_p=0.9,
            repetition_penalty=1.1
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the response part
    response = response.split("### Response:")[-1].strip()
    return response

# Test with sample instructions
test_instructions = [
    "Explain the concept of artificial intelligence in simple terms.",
    "Write a short poem about nature.",
    "What are the benefits of exercise?"
]

print("\n" + "="*50)
print("TESTING THE FINE-TUNED MODEL")
print("="*50)

for i, instruction in enumerate(test_instructions, 1):
    print(f"\nTest {i}:")
    print(f"Instruction: {instruction}")
    response = generate_response(instruction)
    print(f"Response: {response}")
    print("-" * 30)

Step 14: Testing the fine-tuned model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



TESTING THE FINE-TUNED MODEL

Test 1:
Instruction: Explain the concept of artificial intelligence in simple terms.
Response: Artificial Intelligence (AI) is the ability of computers to perform tasks typically associated with human intelligence, such as learning and problem-solving. AI systems can be trained to recognize patterns, make decisions, and even generate creative ideas. AI has revolutionized various industries, including healthcare, finance, and transportation. It has enabled automation, improved efficiency, and enhanced decision-making capabilities. However, there are ethical and societal implications of AI, which should be considered when developing and implementing these systems.
------------------------------

Test 2:
Instruction: Write a short poem about nature.
Response: Roses are red,
Violets are blue,
The sun shines down on me,
And I can't wait to get outside and play!
------------------------------

Test 3:
Instruction: What are the benefits of exercise?
Response: Ex

In [15]:
# Step 15: Optional - Save to Google Drive
print("\nStep 15: Optional - Save to Google Drive")
print("Uncomment the following lines to save the model to Google Drive:")
print("""
# from google.colab import drive
# drive.mount('/content/drive')
#
# import shutil
# shutil.copytree(OUTPUT_DIR, '/content/drive/MyDrive/mistral-7b-alpaca-lora')
# print("Model saved to Google Drive!")
""")

print("\n" + "="*50)
print("FINE-TUNING COMPLETE!")
print("="*50)
print(f"Model saved to: {OUTPUT_DIR}")
print("You can now use this model for inference or further training.")
print("The model has been fine-tuned on 5% of the Alpaca dataset using LoRA.")


Step 15: Optional - Save to Google Drive
Uncomment the following lines to save the model to Google Drive:

# from google.colab import drive
# drive.mount('/content/drive')
#
# import shutil
# shutil.copytree(OUTPUT_DIR, '/content/drive/MyDrive/mistral-7b-alpaca-lora')
# print("Model saved to Google Drive!")


FINE-TUNING COMPLETE!
Model saved to: ./mistral-7b-alpaca-lora
You can now use this model for inference or further training.
The model has been fine-tuned on 5% of the Alpaca dataset using LoRA.
